In [2]:

import numpy as np
from PIL import Image, ImageFilter

def detect_flag_edges(image_path):
    """
    Detects the edges of the flag, removes wrinkles, crops the clean flag, and determines
    if the flag is Indonesia's or Poland's.

    Parameters:
        image_path (str): Path to the flag image.

    Returns:
        str: "Indonesia" or "Poland"
    """
    # Step 1: Open the image and convert to grayscale
    img = Image.open(image_path).convert("L")  # Grayscale
    img = img.filter(ImageFilter.FIND_EDGES)  # Edge detection filter

    # Convert to numpy array
    img_data = np.array(img)
    height, width = img_data.shape

    # Step 2: Find the bounding box of the flag (remove wrinkles)
    flag_rows = np.any(img_data > 50, axis=1)  # Threshold edges in rows
    flag_cols = np.any(img_data > 50, axis=0)  # Threshold edges in columns

    top, bottom = np.where(flag_rows)[0][[0, -1]]  # First and last rows with edges
    left, right = np.where(flag_cols)[0][[0, -1]]  # First and last columns with edges

    # Crop the clean flag area
    flag_area = Image.open(image_path).crop((left, top, right, bottom)).convert("RGB")
    flag_data = np.array(flag_area)
    flag_height, flag_width, _ = flag_data.shape

    # Step 3: Divide the flag into two equal halves (top and bottom)
    top_half = flag_data[:flag_height // 2, :, :]
    bottom_half = flag_data[flag_height // 2:, :, :]

    # Step 4: Calculate the average color for top and bottom halves
    top_avg = np.mean(top_half, axis=(0, 1))  # [R, G, B] for top
    bottom_avg = np.mean(bottom_half, axis=(0, 1))  # [R, G, B] for bottom

    # Step 5: Determine flag based on red dominance
    top_redness = top_avg[0] - top_avg[1] - top_avg[2]
    bottom_redness = bottom_avg[0] - bottom_avg[1] - bottom_avg[2]

    if top_redness > bottom_redness:
        return "Indonesia"  # Red is on top
    else:
        return "Poland"  # White is on top

# Example Usage
image_path = "indonesiaf.png"  # Replace with your image path
result = detect_flag_edges(image_path)
print(f"The flag is of: {result}")



The flag is of: Indonesia
